In [7]:
import cv2 as cv
import pydicom
from pathlib import Path
import randomizer
import natsort
import numpy as np


pathPrefix = 'F:\\rsna-miccai-brain-tumor-radiogenomic-classification\\train\\'

# cleanedData = Path('F:\\brain-tumor-target\\train')
originalData = Path('F:\\rsna-miccai-brain-tumor-radiogenomic-classification\\train')

newData = Path('F:\\brain-tumor-target-2\\train') 

# for patient in cleanedData.iterdir():
#     for type in patient.iterdir():
#         for file in type.iterdir():
#             source = Path(pathPrefix + patient.name + "\\" + type.name + "\\" + file.name)
#             Path("F:\\brain-tumor-target-2\\train\\"  + patient.name + "\\" + type.name + "\\").mkdir(parents=True, exist_ok=True)
#             target = Path("F:\\brain-tumor-target-2\\train\\"  + patient.name + "\\" + type.name + "\\" + file.name)
#             target.symlink_to(source)


In [8]:
testCase = [50, 75, 100]

for i in range(5, 8):
    for patient in newData.iterdir():
        for type in patient.iterdir():
            for file in type.iterdir():
                source = Path(pathPrefix + patient.name + "\\" + type.name + "\\" + file.name)
                Path(f"F:\\brain-tumor-target-{i}\\train\\"  + patient.name + "\\" + type.name + "\\").mkdir(parents=True, exist_ok=True)
                target = Path(f"F:\\brain-tumor-target-{i}\\train\\"  + patient.name + "\\" + type.name + "\\" + file.name)
                dicom = pydicom.dcmread(str(source))
                npdicom = dicom.pixel_array
                if cv.countNonZero(npdicom) >= testCase[i - 5]:
                    target.symlink_to(source)

In [3]:
# cleanedData = Path('F:\\brain-tumor-target-4\\train') 
# 
# minFlair, minTw, minTwce, minT2 = 1000, 1000, 1000, 1000
# a, b, c, d = "", "", "", ""
# for patient in cleanedData.iterdir():
#     for type in patient.iterdir():
#         number = len(list(type.iterdir()))
#         if type.name == "FLAIR":
#             if number < minFlair and number != 0:
#                 minFlair = number
#                 a = patient.name
#         if type.name == "T1w":
#             if number < minTw and number != 0:
#                 minTw = number
#                 b = patient.name
#         if type.name == "T1wCE":
#             if number < minTwce and number != 0:
#                 minTwce = number
#                 c = patient.name
#         if type.name == "T2w":
#             if number < minT2 and number != 0:
#                 minT2 = number
#                 d = patient.name
# 
# minFlair, minTw, minTwce, minT2


In [4]:
from pipeline import Crop

def transformationPipeline(dataList):
    newList = list()
    for item in dataList:
        dicom = pydicom.dcmread(str(item))
        npdicom = dicom.pixel_array
        # crop the image
        cropped = Crop(npdicom)
        newList.append(cropped)
    return np.array(newList)


In [5]:
# newData = Path('F:\\brain-tumor-target-2\\train')
# zero = list(newData.iterdir())[0]
# print(zero.name)
# for type in zero.iterdir():
#     print(type.name)
#     liste = randomizer.uniformTemporal(natsort.natsorted(list(type.iterdir())), 14)
#     print(transformationPipeline(liste))
    

In [6]:
newData = Path('F:\\brain-tumor-target-2\\train')
lastSuccessfulPatient = ""

In [7]:
def translate(tx, ty, image):
    _, _, N, M = image.shape
    image_translated = np.zeros_like(image)
    image_translated[:,:,max(tx,0):M+min(tx,0), max(ty,0):N+min(ty,0)] = image[:,:,-min(tx,0):M-max(tx,0), -min(ty,0):N-max(ty,0)]
    return image_translated

In [8]:
typeList = ["FLAIR", "T1w", "T1wCE", "T2w"]


continued = True


for patient in newData.iterdir():
    if patient.name != lastSuccessfulPatient:
        if continued:
            continue
    else:
        continued = False

    if patient.name == "numpys":
        continue
    
    patientData = [None, None, None, None]
    path = Path(str(newData) + f"\\numpys\\{patient.name}.npy")
    udpath = Path(str(newData) + f"\\numpys\\{patient.name}ud.npy")
    lrpath = Path(str(newData) + f"\\numpys\\{patient.name}lr.npy")
    rot90path = Path(str(newData) + f"\\numpys\\{patient.name}rot90.npy")
    rot270path = Path(str(newData) + f"\\numpys\\{patient.name}rot270.npy")
    ltrpath = Path(str(newData) + f"\\numpys\\{patient.name}ltr.npy")
    rtrpath = Path(str(newData) + f"\\numpys\\{patient.name}rtr.npy")
    utrpath = Path(str(newData) + f"\\numpys\\{patient.name}utr.npy")
    dtrpath = Path(str(newData) + f"\\numpys\\{patient.name}dtr.npy")
    
    for type in patient.iterdir():

        typeName = type.name
        print(patient.name, type.name)
        fileList = randomizer.uniform_temporal_subsample(natsort.natsorted(list(type.iterdir())), 14)  
        index = typeList.index(typeName)
        patientData[index] = transformationPipeline(fileList)

    for item in patientData:
        if item is None:
            item = np.zeros((15, 224, 224))

    for array in patientData:
        print(array.shape)
            
    nparray = np.array(patientData)
    udflip = np.flipud(nparray)
    lrflip = np.fliplr(nparray)
    rot90 = np.rot90(nparray)
    rot270 = np.fliplr(rot90)

    ltr = translate(-5, 0, nparray)
    rtr = translate(5, 0, nparray)
    utr = translate(0, -5, nparray)
    dtr = translate(0, 5, nparray)

    np.save(path, nparray)
    np.save(udpath, udflip)
    np.save(lrpath, lrflip)
    np.save(rot90path, rot90)
    np.save(rot270path, rot270)
    np.save(ltrpath, ltr)
    np.save(rtrpath, rtr)
    np.save(utrpath, utr)
    np.save(dtrpath, dtr)
    lastSuccessfulPatient = patient.name

00000 FLAIR
00000 T1w
00000 T1wCE
00000 T2w
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
00002 FLAIR
00002 T1w
00002 T1wCE
00002 T2w
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
00003 FLAIR
00003 T1w
00003 T1wCE
00003 T2w
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
00005 FLAIR
00005 T1w
00005 T1wCE
00005 T2w
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
00006 FLAIR
00006 T1w
00006 T1wCE
00006 T2w
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
00008 FLAIR
00008 T1w
00008 T1wCE
00008 T2w
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
00009 FLAIR
00009 T1w
00009 T1wCE
00009 T2w
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
00011 FLAIR
00011 T1w
00011 T1wCE
00011 T2w
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
00012 FLAIR
00012 T1w
00012 T1wCE
00012 T2w
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
(14, 224, 224)
00014 FLAIR
00014 T1w
00014 T1wCE
00014 T2w
(14, 224, 224)
(14, 

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [9]:
lastSuccessfulPatient

'00035'